In [ ]:
!git clone https://github.com/arpitbansal297/Universal-Guided-Diffusion.git

Cloning into 'Universal-Guided-Diffusion'...
remote: Enumerating objects: 520, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 520 (delta 64), reused 56 (delta 56), pack-reused 443 (from 1)
Receiving objects: 100% (520/520), 78.85 MiB | 16.72 MiB/s, done.
Resolving deltas: 100% (170/170), done.


In [ ]:
# Step 1: Install required packages
!pip install torch torchvision diffusers transformers
!pip install git+https://github.com/openai/CLIP.git
!pip install accelerate

import torch
import torch.nn.functional as F
from diffusers import StableDiffusionPipeline
from torchvision import transforms
import clip
from PIL import Image

# Step 2: Set up device (GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Step 3: Load Stable Diffusion and CLIP models
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16 if device == "cuda" else torch.float32).to(device)
clip_model, preprocess = clip.load("ViT-B/32", device=device)


  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-h7yqjte0
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-h7yqjte0
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Step 4: Define helper functions for forward and backward universal guidance
def forward_diffusion(image, noise_schedule, t):
    alpha_t = noise_schedule[t]**2
    noise = torch.randn_like(image).to(device)
    noisy_image = torch.sqrt(alpha_t) * image + torch.sqrt(1 - alpha_t) * noise
    return noisy_image, noise

def reverse_diffusion(noisy_image, model, num_steps):
    for t in range(num_steps - 1, -1, -1):
        noisy_image = model(noisy_image, t)
    return noisy_image

# Apply forward guidance using CLIP
def forward_guidance(denoised_image, target_text, guidance_strength=10.0):
    # Ensure the image tensor is properly detached and gradient enabled
    denoised_image = denoised_image.requires_grad_()

    # Convert to PIL and use CLIP's preprocessing pipeline
    denoised_image_pil = transforms.ToPILImage()(denoised_image.squeeze(0).cpu().clamp(0, 1))
    image_input = preprocess(denoised_image_pil).unsqueeze(0).to(device)
    text_input = clip.tokenize([target_text]).to(device)

    # Get the image and text features
    image_features = clip_model.encode_image(image_input)
    text_features = clip_model.encode_text(text_input)

    # Compute cosine similarity loss
    loss = -F.cosine_similarity(image_features, text_features).mean()

    # Compute gradients
    guidance_direction = torch.autograd.grad(loss, denoised_image, retain_graph=True)[0]
    guided_image = denoised_image - guidance_strength * guidance_direction
    return guided_image

# Backward guidance optimization
def backward_guidance(denoised_image, target_prompt, steps=10):
    optimizer = torch.optim.Adam([denoised_image.requires_grad_()], lr=0.01)
    for _ in range(steps):
        optimizer.zero_grad()
        guidance_image = forward_guidance(denoised_image, target_prompt)
        loss = F.mse_loss(guidance_image, denoised_image)
        loss.backward()
        optimizer.step()
    return denoised_image

# Self-recurrence to improve generation diversity
def self_recurrence(noisy_image, diffusion_model, recurrence_steps=4):
    for _ in range(recurrence_steps):
        noise = torch.randn_like(noisy_image).to(device)
        noisy_image = diffusion_model(noisy_image) + noise
    return noisy_image

# Step 5: Main function to apply the universal guidance algorithm
def universal_guided_diffusion(prompt, num_steps=50):
    # Generate initial random noise image
    noisy_image = torch.randn((1, 3, 512, 512)).to(device)
    noise_schedule = torch.linspace(1, 0, num_steps).to(device)

    for t in range(num_steps):
        # Generate denoised image using Stable Diffusion
        denoised_image = pipe(prompt=prompt, num_inference_steps=num_steps)["images"][0]
        denoised_tensor = transforms.ToTensor()(denoised_image).unsqueeze(0).to(device)

        # Apply forward guidance
        guided_image = forward_guidance(denoised_tensor, prompt)

        # Optimize with backward guidance
        optimized_image = backward_guidance(guided_image, prompt)

        # Explore noise manifold with self-recurrence
        noisy_image = self_recurrence(optimized_image, pipe.unet, recurrence_steps=4)

    return noisy_image

# Step 6: Generate image using universal guidance
prompt = "A photograph of a cat sitting on a couch"
generated_image = universal_guided_diffusion(prompt)

# Convert the tensor to a PIL image for display and saving
generated_image_pil = transforms.ToPILImage()(generated_image.squeeze(0).cpu().clamp(0, 1))
generated_image_pil.show()

# Save the result
generated_image_pil.save("generated_cat_image.png")

  0%|          | 0/50 [00:00<?, ?it/s]

RuntimeError: One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.